# Eliminacion gaussiana

Se utiliza numpy unicamente para instanciar arreglos

In [30]:
import numpy as np
from numba import jit
import time

eps_trid = 1e-6

In [31]:
@jit
def restar_multiplo_de_fila(A, i, j, escalar):
    for col in range(len(A[i])):
        A[i][col] -= A[j][col] * escalar
    return A

/tmp/ipykernel_6913/3469964432.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


### Algoritmo

El algoritmo realiza dos pasos.

1. Primero se diagonaliza la matriz de entrada. Las operaciones de multiplicacion por escalar y resta de filas se realiza tanto para la matriz como para el vector de terminos independientes
2. Se calculan los valores de la solucion desde la ultima fila hacia la primera

Si en algun momento el pivot, es decir el elemento que se usa para diagonalizar las filas posteriores, es 0, el algoritmo levanta una excepcion.

In [32]:
@jit
def diagonalizar_columna(A, b, i):
    pivot = A[i][i]
    if (pivot == 0):
        raise Exception('No puede resolverse')
    elif (abs(pivot) < eps_trid):
        print(F'Division por numero cercano a 0 puede ocasionar error numérico: {pivot}')
    for fila in range(i + 1, len(A)):
        if (A[fila][i] != 0):
            escalar = A[fila][i]/pivot
            restar_multiplo_de_fila(A, fila, i, escalar)
            b[fila] = b[fila] - (b[i] * escalar)

/tmp/ipykernel_6913/559068186.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [33]:
@jit
def resolver_sistema_diagonal(A, b):
    solucion = np.zeros(len(b))
    for i in range(len(solucion)-1, -1, -1):
        suma_variables_posteriores = 0
        for j in range (i + 1, len(solucion)):
            suma_variables_posteriores += A[i][j] * solucion[j]

        if (A[i][i] != 0):
            solucion[i] = (b[i] - suma_variables_posteriores)/A[i][i]
        else:
            solucion[i] = 0
    return solucion

/tmp/ipykernel_6913/2877216679.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [34]:
@jit
def permutar(matriz, i, j):
    for col in range(len(matriz)):
        temp = matriz[i][col]
        matriz[i][col] = matriz[j][col]
        matriz[j][col] = temp
    return matriz

/tmp/ipykernel_6913/2261804321.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [35]:
@jit
def permutar_vector(vector, i, j):
    temp = vector[i]
    vector[i] = vector[j]
    vector[j] = temp
    return vector

/tmp/ipykernel_6913/3862351391.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [36]:
@jit
def eliminacion_gaussiana_SP(A, b):
    for i in range(len(A) - 1):
        diagonalizar_columna(A, b, i)
    return resolver_sistema_diagonal(A, b)

/tmp/ipykernel_6913/2166191619.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [37]:
@jit
def eliminacion_gaussiana_CP(A, b):
    A_res = np.copy(A)
    b_res = np.copy(b)
    start_time = time.time()
    # Diagonalizar
    for i in range(len(A_res) - 1):
        fila_pivot = i
        for j in range(i + 1, len(A_res)):
            if (A_res[j][i] > A_res[fila_pivot][i]):
                fila_pivot = j
        if (fila_pivot != i):
            permutar(A_res, i, fila_pivot)
            permutar_vector(b_res, i, fila_pivot)
        if (A_res[i][i] != 0):
            diagonalizar_columna(A_res, b_res, i)
    return [resolver_sistema_diagonal(A_res, b_res), (time.time() - start_time)]

/tmp/ipykernel_6913/572586751.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


# Gauss Tri-diagonal tradicional

In [38]:
@jit
def EG_tridiagonal(a,b,c,d):
    a_res = np.copy(a)
    b_res = np.copy(b)
    c_res = np.copy(c)
    d_res = np.copy(d)
    
    start_time = time.time()
    a_sol, b_sol, c_sol, d_sol = eliminacion_gaussiana_tridiagonal(a_res, b_res, c_res, d_res)
    res = sustitucion_hacia_atras(b_sol, c_sol, d_sol)
    
    return [res, (time.time() - start_time)]

/tmp/ipykernel_6913/2280620609.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [39]:
# Dados los vectores a, b, c (matriz tri-diagonal) y d (vector independiente)
# devuelve los vectores resultantes de aplicarles eliminacion gaussiana
@jit
def eliminacion_gaussiana_tridiagonal(a, b, c, d):
    n = b.shape[0]
    for i in range(n-1):
        if abs(b[i]) < eps_trid:
            raise Exception("Gauss no aplicable")
        m = a[i+1]/b[i]
        a[i+1] = 0.0
        b[i+1] = b[i+1] - (m * c[i])
        d[i+1] = d[i+1] - (m * d[i])
    if abs(b[n-1]) < eps_trid:
        if abs(d[n-1]) < eps_trid:
            raise Exception("Sistema con infinitas soluciones")
        else:
            raise Exception("Sistema sin soluciones")
    return a, b, c, d

/tmp/ipykernel_6913/2888417156.py:3: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [40]:
# Dados los vectores b, c y d resultantes de aplicar eliminacion gaussiana (el vector a no se utiliza),
#   devuelve el vector resultado x
@jit
def sustitucion_hacia_atras(b_res, c_res, d_res):
    n = b_res.shape[0]
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        if i == (n-1):
            x[i] = d_res[i] / b_res[i] # b_res[i] == 0 ??
        else:
            x[i] = (d_res[i] - (x[i+1] * c_res[i])) / b_res[i]
    return x

/tmp/ipykernel_6913/47674028.py:3: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


# Gauss Tri-diagonal con pre-computo

In [41]:
# Dados los vectores a, b y c (matriz tri-diagonal)
# devuelve los vectores resultantes de aplicarles eliminacion gaussiana
# y un vector que contiene los multiplicadores pre-calculados a aplicar sobre el vector independiente d
@jit
def precomputo_eliminacion_gaussiana_tridiagonal(a, b, c):
    n = b.shape[0]
    a_res = np.copy(a)
    b_res = np.copy(b)
    c_res = np.copy(c)
    d_precomputo = np.zeros(n)
    for i in range(n-1):
        if abs(b_res[i]) < eps_trid:
            raise Exception("Gauss no aplicable")
        m = a_res[i+1]/b_res[i]
        a_res[i+1] = 0.0
        b_res[i+1] = b_res[i+1] - (m * c_res[i])
        d_precomputo[i+1] = m
    if abs(b_res[n-1]) < eps_trid:
        raise Exception("Sistema con ninguna o infinitas soluciones")
    return a_res, b_res, c_res, d_precomputo

/tmp/ipykernel_6913/2143533843.py:4: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [42]:
# Dado el vector independiente d,
# devuelve el resultado de aplicarle los multiplicadores pre-calculados
# Funcion auxiliar utilizada en sustitucion_hacia_atras_con_precomputo
@jit
def aplicar_precomputo(d, d_precomputo):
    n = d.shape[0]
    d_res = np.zeros(n)
    for i in range(n):
        if i == 0:
            d_res[i] = d[i]
        else:
            d_res[i] = d[i] - (d_precomputo[i] * d_res[i-1])
    return d_res

/tmp/ipykernel_6913/3472278689.py:4: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [43]:
# Dados los vectores b y c resultantes de aplicar gauss, el vector con los multiplicadores pre-calculados y el vector independiente d,
# devuelve el vector resultado x
@jit
def sustitucion_hacia_atras_con_precomputo(b_res, c_res, d_precomputo, d):
    start_time = time.time()
    d_res = aplicar_precomputo(d, d_precomputo)
    n = b_res.shape[0]
    res = np.zeros(n)
    for i in range(n-1, -1, -1):
        if i == (n-1):
            res[i] = d_res[i] / b_res[i] # TODO: si b_res[i] == 0 ??
        else:
            res[i] = (d_res[i] - (res[i+1] * c_res[i])) / b_res[i]
    return [res, (time.time() - start_time)]

/tmp/ipykernel_6913/28558854.py:3: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
